In [54]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import statsmodels.api as sm

import matplotlib.pyplot as plt
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

### Data Prep

In [2]:
load_dotenv()
file_path = os.environ['data_path']

In [3]:
all_files = os.listdir(file_path)
all_files

['PRSA_Data_Aotizhongxin_20130301-20170228.csv',
 'PRSA_Data_Changping_20130301-20170228.csv',
 'PRSA_Data_Dingling_20130301-20170228.csv',
 'PRSA_Data_Dongsi_20130301-20170228.csv',
 'PRSA_Data_Guanyuan_20130301-20170228.csv',
 'PRSA_Data_Gucheng_20130301-20170228.csv',
 'PRSA_Data_Huairou_20130301-20170228.csv',
 'PRSA_Data_Nongzhanguan_20130301-20170228.csv',
 'PRSA_Data_Shunyi_20130301-20170228.csv',
 'PRSA_Data_Tiantan_20130301-20170228.csv',
 'PRSA_Data_Wanliu_20130301-20170228.csv',
 'PRSA_Data_Wanshouxigong_20130301-20170228.csv']

In [4]:
df = pd.read_csv(file_path+all_files[0])
df.columns = df.columns.str.lower()
df['ts'] = pd.to_datetime(df[['year','month','day','hour']])
df['date'] = pd.to_datetime(df[['year','month','day']])
print(df.shape)
df.head()

(35064, 20)


,no,year,month,day,hour,pm2.5,pm10,so2,no2,co,o3,temp,pres,dewp,rain,wd,wspm,station,ts,date
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin,2013-03-01 00:00:00,2013-03-01
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin,2013-03-01 01:00:00,2013-03-01
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin,2013-03-01 02:00:00,2013-03-01
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin,2013-03-01 03:00:00,2013-03-01
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin,2013-03-01 04:00:00,2013-03-01


In [5]:
# Changing units for CO - the data is microgram whereas the thresholds from the video are in mg
df['co'] = df['co']/1000

In [6]:
# AQI Calculation
df_aqi = pd.read_excel('aqi_ranges.xlsx')
df_aqi.columns = df_aqi.columns.str.lower()

### Calculating the sub-indexes for all pollutants
pollutants = ['pm2.5','pm10','so2','no2','co','o3']
df_c = df.copy()

for x in pollutants:
    col_to_partition = df[x].tolist()
    part_ranges = df_aqi[x+'_min'].tolist() + [1000000]
    
    aqi_min = pd.cut(col_to_partition, bins=part_ranges, labels=df_aqi['aqi_min'].tolist())
    aqi_max = pd.cut(col_to_partition, bins=part_ranges, labels=df_aqi['aqi_max'].tolist())
    pol_min = pd.cut(col_to_partition, bins=part_ranges, labels=df_aqi[x+'_min'].tolist())
    pol_max = pd.cut(col_to_partition, bins=part_ranges, labels=df_aqi[x+'_max'].tolist())
    df_temp = pd.DataFrame(data={
        'i_min':aqi_min,
        'i_max':aqi_max,
        'c_min':pol_min,
        'c_max':pol_max,
        'c_obs':col_to_partition},
        dtype='float64') # Change if handling lots of data
    c = x + '_si' # Pollutant sub-index column
    df_temp[c] = df_temp['c_obs'] - df_temp['c_min']
    df_temp[c] = df_temp[c]*(df_temp['i_max'] - df_temp['i_min'])
    df_temp['den'] = (df_temp['c_max'] - df_temp['c_min'])
    df_temp.loc[df_temp['den'] == 0,'den'] = 1 # To avoid zero division
    df_temp[c] = df_temp[c]/df_temp['den']
    df_temp[c] = df_temp[c] + df_temp['i_min']
    df_c[c] = df_temp[c]

# The AQI is the max of all the sub-indexes
pol_si_names = [i+'_si' for i in pollutants]
df_c['aqi'] = df_c[pol_si_names].apply(max,axis=1)
df_c['dominant'] = df_c[pol_si_names].fillna(-1).apply(lambda x: pol_si_names[np.argmax(x)],axis=1)

print(df_c.shape)
df_c.head()

(35064, 28)


,no,year,month,day,hour,pm2.5,pm10,so2,no2,co,o3,temp,pres,dewp,rain,wd,wspm,station,ts,date,pm2.5_si,pm10_si,so2_si,no2_si,co_si,o3_si,aqi,dominant
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,0.3,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin,2013-03-01 00:00:00,2013-03-01,6.666667,4.0,5.00,8.75,15.0,77.0,77.0,o3_si
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,0.3,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin,2013-03-01 01:00:00,2013-03-01,13.333333,8.0,5.00,8.75,15.0,77.0,77.0,o3_si
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,0.3,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin,2013-03-01 02:00:00,2013-03-01,11.666667,7.0,6.25,12.50,15.0,73.0,73.0,o3_si
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,0.3,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin,2013-03-01 03:00:00,2013-03-01,10.000000,6.0,13.75,13.75,15.0,72.0,72.0,o3_si
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,0.3,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin,2013-03-01 04:00:00,2013-03-01,5.000000,3.0,15.00,15.00,15.0,72.0,72.0,o3_si


In [7]:
# Wind Direction
# https://www.quora.com/What-direction-is-a-NNW-wind-coming-from
# Converting wind direction to angles
# Need to doc
wd = df.wd.dropna().unique().tolist()
wd_orig = wd.copy()
angles = {} # Measured clockwise from North
d = {'N':1,'S':-1,'E':-1,'W':1}
vert_angles = {'N':0,'S':180}
hor_angles = {'E':45, 'W':-45}

wd_angles = dict(zip(wd,[0]*len(wd)))
for direction in wd:
    dd = 0
    if len(direction) == 1:
        dd += vert_angles.get(direction,0) + hor_angles.get(direction,0)*2
    if len(direction) == 2:
        dd += vert_angles.get(direction[0],0) + hor_angles.get(direction[1],0)*d.get(direction[0],1)
    if len(direction) == 3:
        dd += vert_angles.get(direction[1],0) + hor_angles.get(direction[2],0)*d.get(direction[1],1)
        dd = dd%360
        inc = (vert_angles.get(direction[0],0) + (hor_angles.get(direction[0],0)*2)%360)/45
        inc = 8 if inc == 0 and dd > 180 else inc
        inc2 = inc - dd/45
        dd += 22.5*inc2

        # print(direction,dd,inc,dd/45,inc2)
        # print(22.5*(((((vert_angles.get(direction[0],0)) + (hor_angles.get(direction[0],0)*2))%360)/45) - (dd/45)))
        # dd += 22.5*(((((vert_angles.get(direction[0],0)) + (hor_angles.get(direction[0],0)*2))%360)/45) - (dd/45))
        # print(direction, ((((vert_angles.get(direction[0],0)) + (hor_angles.get(direction[0],0)*2))%360)/45))
        # print(vert_angles.get(direction[1],0) + hor_angles.get(direction[2],0)*d.get(direction[1],1))

    if dd < 0:
        dd = 360 + dd
    wd_angles[direction] = dd
wd_angles = pd.DataFrame({'wd':wd_angles.keys(),'wd_angle':wd_angles.values()})

df_c = df_c.merge(wd_angles,on='wd',how='left')

print(df_c.shape)
df_c.head()
print(df_c.shape)

(35064, 29)
(35064, 29)


In [8]:
covars = ['temp','pres','rain','dewp','wspm']

### Detecting Anomalies - Static Thresholds (Category 1) - Suffix _st

In [9]:
df_aqi

,aqi_category,aqi_min,aqi_max,pm2.5_min,pm2.5_max,pm10_min,pm10_max,so2_min,so2_max,no2_min,no2_max,co_min,co_max,o3_min,o3_max
0,Good,0,50,0,30,0,50,0,40,0,40,0.0,1,0,50
1,Satisfactory,51,100,31,60,51,100,41,80,41,80,1.1,2,51,100
2,Moderately polluted,101,200,61,90,101,250,81,380,81,180,2.1,10,101,168
3,Poor,201,300,91,120,251,350,381,800,181,280,10.0,17,169,208
4,Very poor,301,400,121,250,351,430,801,1600,281,400,17.0,34,209,748
5,Severe,401,500,250,10000,430,10000,1600,10000,400,10000,34.0,10000,748,10000


In [10]:
# The threshold would be subject to the requirements, such as how high the sensor values before some action can be taken,
# such as a warning issued to the citizens to stay indoors
static_thresholds = df_aqi.loc[df_aqi['aqi_category']=='Severe', [x+'_min' for x in pollutants]]
static_thresholds = dict(zip(pollutants,[static_thresholds.loc[:,x+'_min'].tolist()[0] for x in pollutants]))
print('Thresholds used: ', static_thresholds)

for p in static_thresholds:
    df_c[p+'_st'] = 0
    breach_condition = df_c[p] >= static_thresholds[p]
    df_c.loc[breach_condition, p+'_st'] = 1

df_c.head()

Thresholds used:  {'pm2.5': 250, 'pm10': 430, 'so2': 1600, 'no2': 400, 'co': 34.0, 'o3': 748}


,no,year,month,day,hour,pm2.5,pm10,so2,no2,co,o3,temp,pres,dewp,rain,wd,wspm,station,ts,date,pm2.5_si,pm10_si,so2_si,no2_si,co_si,o3_si,aqi,dominant,wd_angle,pm2.5_st,pm10_st,so2_st,no2_st,co_st,o3_st
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,0.3,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin,2013-03-01 00:00:00,2013-03-01,6.666667,4.0,5.00,8.75,15.0,77.0,77.0,o3_si,337.5,0,0,0,0,0,0
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,0.3,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin,2013-03-01 01:00:00,2013-03-01,13.333333,8.0,5.00,8.75,15.0,77.0,77.0,o3_si,0.0,0,0,0,0,0,0
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,0.3,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin,2013-03-01 02:00:00,2013-03-01,11.666667,7.0,6.25,12.50,15.0,73.0,73.0,o3_si,337.5,0,0,0,0,0,0
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,0.3,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin,2013-03-01 03:00:00,2013-03-01,10.000000,6.0,13.75,13.75,15.0,72.0,72.0,o3_si,315.0,0,0,0,0,0,0
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,0.3,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin,2013-03-01 04:00:00,2013-03-01,5.000000,3.0,15.00,15.00,15.0,72.0,72.0,o3_si,0.0,0,0,0,0,0,0


In [11]:
df_c[[x+'_st' for x in pollutants]].mean()

pm2.5_st    0.047713
pm10_st     0.010552
so2_st      0.000000
no2_st      0.000000
co_st       0.000000
o3_st       0.000000
dtype: float64

### Detecting Anomalies - Outlier Detection (1-D) - Dynamic Thresholds (Category 3) - Suffix _dt
Methodology based on EDA - R3

In [12]:
df_m = df_c.groupby('date')[pollutants+covars].mean()
df_m = df_m.reset_index()
df_m.head()

,date,pm2.5,pm10,so2,no2,co,o3,temp,pres,rain,dewp,wspm
0,2013-03-01,7.125000,10.750000,11.708333,22.583333,0.429167,63.875000,1.391667,1026.875000,0.0,-18.745833,3.254167
1,2013-03-02,30.750000,42.083333,36.625000,66.666667,0.824917,29.750000,0.616667,1026.850000,0.0,-15.937500,1.479167
2,2013-03-03,76.916667,120.541667,61.291667,81.000000,1.620625,19.125000,5.566667,1014.608333,0.0,-12.316667,1.658333
3,2013-03-04,22.708333,44.583333,22.869565,46.956522,0.617391,53.750000,9.962500,1017.650000,0.0,-11.683333,2.404167
4,2013-03-05,148.875000,183.791667,93.875000,132.833333,2.357958,68.458333,6.291667,1010.900000,0.0,-7.525000,1.129167


In [13]:
# A running z-test which checks the average value of each sensor each day compared to the past 30 days
period = 30
rm_names = [x + '_rm' for x in pollutants]
rs_names = [x + '_rs' for x in pollutants]
dt_names = [x + '_dt' for x in pollutants]
df_m = df_m.sort_values('date')

df_m[rm_names] = df_m[pollutants].rolling(period, min_periods=1).mean()
df_m[rs_names] = df_m[pollutants].rolling(period, min_periods=1).std()

for p in pollutants:
    df_m[p + '_dtz'] = df_m[[p,p+'_rm',p+'_rs']].apply(lambda x: (x[0]-x[1])/x[2],axis=1)

# Separate loops to keep similar columns together
for p in pollutants:
    df_m[p + '_dt'] = (~df_m[p + '_dtz'].between(-3,3)).astype(int)
    df_m.loc[df_m[p].isnull(),p + '_dt'] = 0
df_m.sample(10)

C:\Users\MishMish\AppData\Local\Temp\ipykernel_23840\2660683365.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_m[p + '_dtz'] = df_m[[p,p+'_rm',p+'_rs']].apply(lambda x: (x[0]-x[1])/x[2],axis=1)
C:\Users\MishMish\AppData\Local\Temp\ipykernel_23840\2660683365.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_m[p + '_dtz'] = df_m[[p,p+'_rm',p+'_rs']].apply(lambda x: (x[0]-x[1])/x[2],axis=1)
C:\Users\MishMish\AppData\Local\Temp\ipykernel_23840\2660683365.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (con

,date,pm2.5,pm10,so2,no2,co,o3,temp,pres,rain,dewp,wspm,pm2.5_rm,pm10_rm,so2_rm,no2_rm,co_rm,o3_rm,pm2.5_rs,pm10_rs,so2_rs,no2_rs,co_rs,o3_rs,pm2.5_dtz,pm10_dtz,so2_dtz,no2_dtz,co_dtz,o3_dtz,pm2.5_dt,pm10_dt,so2_dt,no2_dt,co_dt,o3_dt
1210,2016-06-23,79.913043,101.173913,5.739130,30.652174,1.030435,109.956522,27.379167,998.800000,0.000,18.404167,1.579167,54.101749,74.026668,7.204490,28.272911,0.738948,121.945929,29.875220,29.669669,6.069744,7.219111,0.274259,31.663130,0.863970,0.914983,-0.241420,0.329578,1.062818,-0.378655,0,0,0,0,0,0
485,2014-06-29,46.916667,46.916667,13.750000,57.333333,1.079167,105.125000,28.445833,999.937500,0.000,17.962500,1.066667,61.066727,66.811171,7.365882,51.577476,0.731093,88.877295,35.039057,47.524452,8.445311,11.931063,0.357935,24.409854,-0.403837,-0.418616,0.755936,0.482426,0.972450,0.665621,0,0,0,0,0,0
978,2015-11-04,212.695652,240.458333,15.333333,150.208333,2.816667,5.833333,9.733333,1019.004167,0.000,4.987500,0.854167,84.933037,104.273611,7.361111,77.552729,1.314257,36.577093,93.038884,90.603735,5.057705,43.326612,0.865544,20.503918,1.373217,1.503081,1.576253,1.676928,1.735799,-1.499409,0,0,0,0,0,0
1105,2016-03-10,14.461538,57.375000,5.625000,16.208333,0.387500,70.260870,2.679167,1026.716667,0.000,-20.875000,3.345833,69.836902,99.285074,19.815102,39.371992,1.344348,48.136870,84.834159,97.624209,19.617301,27.860381,1.352622,19.132678,-0.652748,-0.429300,-0.723346,-0.831419,-0.707403,1.156346,0,0,0,0,0,0
1203,2016-06-16,23.285714,41.954545,2.083333,38.125000,0.491667,82.695652,26.841667,993.962500,0.000,13.266667,1.479167,54.534118,72.076932,9.524417,28.979433,0.727831,121.879531,28.715984,29.543253,6.974624,7.930182,0.282375,28.394199,-1.088189,-1.019603,-1.066880,1.153261,-0.836349,-1.379996,0,0,0,0,0,0
889,2015-08-07,95.291667,103.291667,3.625000,48.250000,1.295833,89.291667,25.200000,1007.866667,2.025,21.833333,1.608333,69.736704,86.063171,4.338628,54.750497,0.863202,115.764869,32.428989,32.672074,2.992596,10.136132,0.220683,36.026277,0.788028,0.527316,-0.238465,-0.641319,1.960417,-0.734830,0,0,0,0,0,0
321,2014-01-16,358.000000,396.958333,134.083333,134.041667,4.941667,3.666667,0.441667,1022.016667,0.000,-8.637500,1.383333,87.329891,125.708333,54.456650,68.956135,1.934669,14.564046,74.349050,85.619772,31.681305,29.093364,1.255404,11.955347,3.640532,3.168077,2.513365,2.237126,2.395244,-0.911507,1,1,0,0,0,0
1438,2017-02-06,11.250000,20.125000,8.875000,28.708333,0.391667,73.583333,2.010417,1027.441667,0.000,-17.825000,2.337500,75.714855,87.925664,23.451120,56.384475,1.256539,53.721360,83.317734,87.812930,19.896754,28.015884,0.899532,18.023279,-0.773723,-0.772103,-0.732588,-0.987873,-0.961469,1.102018,0,0,0,0,0,0
13,2013-03-14,95.166667,107.291667,51.916667,70.375000,1.337292,68.916667,3.816667,1021.400000,0.000,-3.587500,1.954167,103.470238,141.646092,53.393045,80.239418,1.586209,55.863095,84.443293,102.658109,34.289127,40.798295,1.004940,22.570571,-0.098333,-0.334649,-0.043057,-0.241785,-0.247694,0.578345,0,0,0,0,0,0
630,2014-11-21,194.041667,241.833333,20.791667,119.791667,2.841667,8.833333,5.920833,1020.425000,0.000,-3.750000,1.000000,99.808793,129.961717,14.328500,81.160877,1.419066,17.441623,90.246144,101.537918,11.130518,38.404766,0.909328,13.401075,1.044176,1.101772,0.580671,1.005885,1.564454,-0.642358,0,0,0,0,0,0


In [14]:
df_m[dt_names].mean()

pm2.5_dt    0.012320
pm10_dt     0.009582
so2_dt      0.015743
no2_dt      0.004107
co_dt       0.008214
o3_dt       0.004107
dtype: float64

In [15]:
df_m[df_m[dt_names].sum(axis=1) > 0]

,date,pm2.5,pm10,so2,no2,co,o3,temp,pres,rain,dewp,wspm,pm2.5_rm,pm10_rm,so2_rm,no2_rm,co_rm,o3_rm,pm2.5_rs,pm10_rs,so2_rs,no2_rs,co_rs,o3_rs,pm2.5_dtz,pm10_dtz,so2_dtz,no2_dtz,co_dtz,o3_dtz,pm2.5_dt,pm10_dt,so2_dt,no2_dt,co_dt,o3_dt
0,2013-03-01,7.125000,10.750000,11.708333,22.583333,0.429167,63.875000,1.391667,1026.875000,0.000000,-18.745833,3.254167,7.125000,10.750000,11.708333,22.583333,0.429167,63.875000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,1
119,2013-06-28,318.000000,329.458333,32.755130,76.704148,2.709600,62.861295,25.204167,995.575000,0.708333,22.016667,1.125000,100.548853,127.506944,15.503348,72.985293,1.229559,77.660369,58.419606,57.895435,10.793216,14.040609,0.700652,43.298544,3.722229,3.488209,1.598391,0.264864,2.112376,-0.341791,1,1,0,0,0,0
174,2013-08-22,117.565217,148.875000,5.375000,102.833333,1.172727,16.714050,23.645833,1003.533333,0.008333,21.725000,0.775000,69.643841,88.656944,7.746407,57.454875,0.804621,77.496428,38.241032,40.815268,4.652697,13.334860,0.332995,34.007373,1.253140,1.475381,-0.509684,3.402995,1.105441,-1.787329,0,0,0,1,0,0
191,2013-09-08,85.541667,118.083333,20.458333,65.291667,1.358333,42.666667,21.029167,1010.900000,0.387500,17.483333,1.045833,64.521618,87.775242,8.425991,56.833138,0.903089,62.946798,36.616698,38.958414,3.930981,14.802649,0.372358,28.350528,0.574056,0.777960,3.060901,0.571420,1.222598,-0.715335,0,0,1,0,0,0
195,2013-09-12,149.000000,198.250000,25.041667,111.541667,1.537500,35.041667,21.350000,1006.000000,0.170833,18.020833,0.962500,63.141063,88.823853,9.686111,59.687433,0.906047,56.885873,36.135379,40.634960,4.928982,17.484838,0.376176,23.505463,2.376035,2.692907,3.115360,2.965669,1.678608,-0.929325,0,0,1,0,0,0
210,2013-09-27,161.750000,210.833333,34.166667,102.625000,1.662500,52.458333,16.587500,1005.920833,0.000000,11.591667,0.862500,60.429167,87.074275,11.851389,58.365790,0.963209,38.313410,37.907573,43.510007,7.030641,18.284325,0.490902,21.101694,2.672839,2.844381,3.174003,2.420609,1.424501,0.670322,0,0,1,0,0,0
211,2013-09-28,241.083333,260.666667,22.415233,85.834612,1.347368,47.774983,15.583333,1005.791667,0.000000,12.729167,0.712500,67.530556,94.143720,12.473563,60.339764,0.986454,38.851743,49.739537,53.192544,7.114378,17.931605,0.492069,21.131163,3.489232,3.130569,1.397405,1.421783,0.733461,0.422279,1,1,0,0,0,0
217,2013-10-04,139.458333,170.750000,46.916667,92.291667,2.120833,31.166667,17.237500,1008.587500,0.000000,13.208333,0.904167,80.491063,105.377053,15.676341,65.620320,1.102871,31.751646,58.597720,59.424726,9.925242,19.466224,0.524765,16.989416,1.006307,1.100097,3.147563,1.370135,1.939844,-0.034432,0,0,1,0,0,0
281,2013-12-07,265.125000,275.833333,82.750000,103.541667,5.166667,1.263158,2.054167,1014.137500,0.000000,-3.208333,1.070833,75.861594,103.931944,33.454167,59.669444,1.643205,19.759634,60.732528,63.619518,20.630464,27.452216,1.084928,15.319808,3.116343,2.702023,2.389468,1.598130,3.247646,-1.207357,1,0,0,0,1,0
298,2013-12-24,219.541667,269.416667,119.291667,124.166667,5.241667,1.500000,-2.495833,1027.708333,0.000000,-9.512500,1.229167,71.832669,94.297222,39.088417,58.691376,1.798667,15.601777,65.075964,68.105622,26.313874,26.893687,1.380902,10.967758,2.269793,2.571292,3.047945,2.434597,2.493298,-1.285748,0,0,1,0,0,0


### Detecting Anomalies - Outlier Detection (N-D) - Static Thresholds (Category 3) - Suffix _cm
Methodology based on EDA - R4

In [16]:
# The general idea is that the existing data can be grouped into clusters
# Any new points which are very far away from the existing clusters may be treated as anomalies
# Existing points too far away from their clusters may also represent anomalies
# Note: The solution presented below can be further refined by finding the optimal number of clusters and removing already identified anomalies
# from the clustering. The scalers can also be given fixed min and max instead of inferring them from the data

# Two versions: With or without PCA

In [17]:
c_data = df_c.groupby('date')[pollutants].mean().fillna(-1).reset_index()

#### Normal Version

In [18]:
# Creating clusters
mm_scaler = MinMaxScaler()
mm_scaler.fit(c_data[pollutants])
t_cols = [x+'_t' for x in pollutants]
c_data[t_cols] = mm_scaler.transform(c_data[pollutants])

km = KMeans(random_state=22) # The optimal number of clusters can be figured out. Arbitrary initialization - n=8 (sklearn default)
km.fit(c_data[t_cols])
c_data['cluster_id'] = km.labels_
c_data.head()

,date,pm2.5,pm10,so2,no2,co,o3,pm2.5_t,pm10_t,so2_t,no2_t,co_t,o3_t,cluster_id
0,2013-03-01,7.125000,10.750000,11.708333,22.583333,0.429167,63.875000,0.015829,0.021486,0.088785,0.132898,0.141374,0.330555,1
1,2013-03-02,30.750000,42.083333,36.625000,66.666667,0.824917,29.750000,0.061856,0.078781,0.262861,0.381318,0.180522,0.156679,6
2,2013-03-03,76.916667,120.541667,61.291667,81.000000,1.620625,19.125000,0.151798,0.222248,0.435191,0.462090,0.259234,0.102542,0
3,2013-03-04,22.708333,44.583333,22.869565,46.956522,0.617391,53.750000,0.046189,0.083352,0.166761,0.270247,0.159994,0.278965,1
4,2013-03-05,148.875000,183.791667,93.875000,132.833333,2.357958,68.458333,0.291988,0.337905,0.662829,0.754184,0.332172,0.353908,3


In [19]:
# Computing distances from the respective centroids
dist = np.array(c_data[t_cols] - km.cluster_centers_[c_data.cluster_id.tolist()])
dist = dist*dist
c_data['d'] = dist.sum(axis=1)

In [20]:
p95 = lambda x: np.percentile(x,95)
p90 = lambda x: np.percentile(x,90)
p5 = lambda x: np.percentile(x,5)
c_data.groupby('cluster_id').d.agg(['mean','std',p5,p90,p95,'count']).reset_index()

,cluster_id,mean,std,<lambda_0>,<lambda_1>,<lambda_2>,count
0,0,0.039455,0.027542,0.011272,0.065909,0.086992,117
1,1,0.023196,0.031202,0.002754,0.037539,0.049364,385
2,2,0.034035,0.032446,0.006225,0.070061,0.099366,221
3,3,0.089078,0.058383,0.025253,0.167519,0.184223,41
4,4,0.054609,0.038522,0.016250,0.097331,0.126461,115
5,5,0.094276,0.110784,0.009654,0.149887,0.252890,43
6,6,0.022676,0.014229,0.005701,0.040486,0.049669,340
7,7,0.037442,0.032506,0.008998,0.070419,0.102670,199


In [21]:
c_data.groupby(lambda _: True).d.aggregate(['mean','std',p5,p90,p95,'count']).reset_index()

,index,mean,std,<lambda_0>,<lambda_1>,<lambda_2>,count
0,True,0.034371,0.038958,0.00512,0.066295,0.101594,1461


In [22]:
t95 = p95(c_data['d'])
t95

np.float64(0.10159402079272813)

In [23]:
c_data['anom_cm'] = 0
c_data.loc[c_data['d']>t95,'anom_cm'] = 1

In [24]:
# c_data[c_data['anom_cm'] == 1][pollutants+['d']].sort_values('d')

#### PCA Version

In [25]:
# Doing the same with PCA - might be easier to visualize
# Targeting 90% explained variance
from sklearn.decomposition import PCA
var_target = 0.9
pca = PCA()
pca.fit(c_data[t_cols])

print(pca.explained_variance_ratio_)
n_comp = sum(pca.explained_variance_ratio_.cumsum() < 0.9) + 1
pca = PCA(n_components=n_comp).fit(c_data[t_cols])
pca_cols = ['pca_' + str(i+1) for i in range(n_comp)]

c_data[pca_cols] = pca.transform(c_data[t_cols])
c_data.head()

[0.58852861 0.25077568 0.08192689 0.04375956 0.02531031 0.00969896]


,date,pm2.5,pm10,so2,no2,co,o3,pm2.5_t,pm10_t,so2_t,no2_t,co_t,o3_t,cluster_id,d,anom_cm,pca_1,pca_2,pca_3
0,2013-03-01,7.125000,10.750000,11.708333,22.583333,0.429167,63.875000,0.015829,0.021486,0.088785,0.132898,0.141374,0.330555,1,0.008399,0,-0.295134,-0.100661,0.069342
1,2013-03-02,30.750000,42.083333,36.625000,66.666667,0.824917,29.750000,0.061856,0.078781,0.262861,0.381318,0.180522,0.156679,6,0.035934,0,0.019098,-0.182715,0.128209
2,2013-03-03,76.916667,120.541667,61.291667,81.000000,1.620625,19.125000,0.151798,0.222248,0.435191,0.462090,0.259234,0.102542,0,0.012337,0,0.263227,-0.133823,0.228975
3,2013-03-04,22.708333,44.583333,22.869565,46.956522,0.617391,53.750000,0.046189,0.083352,0.166761,0.270247,0.159994,0.278965,1,0.022487,0,-0.132480,-0.095857,0.086555
4,2013-03-05,148.875000,183.791667,93.875000,132.833333,2.357958,68.458333,0.291988,0.337905,0.662829,0.754184,0.332172,0.353908,3,0.152022,1,0.514699,0.214526,0.352351


In [26]:
km_pca = KMeans(random_state=23) # The optimal number of clusters can be figured out. Arbitrary initialization - n=8 (sklearn default)
km_pca.fit(c_data[pca_cols])
c_data['cluster_id_pca'] = km_pca.labels_

In [27]:
# Computing distances from the respective centroids
dist = np.array(c_data[pca_cols] - km_pca.cluster_centers_[c_data.cluster_id_pca.tolist()])
dist = dist*dist
c_data['d_pca'] = dist.sum(axis=1)# Computing distances from the respective centroids

In [28]:
c_data.groupby(lambda _: True).d_pca.aggregate(['mean','std',p5,p90,p95,'count']).reset_index()

,index,mean,std,<lambda_0>,<lambda_1>,<lambda_2>,count
0,True,0.024491,0.032622,0.002196,0.046844,0.071437,1461


In [29]:
c_data.groupby('cluster_id').d_pca.agg(['mean','std',p5,p90,p95,'count']).reset_index()

,cluster_id,mean,std,<lambda_0>,<lambda_1>,<lambda_2>,count
0,0,0.030187,0.024493,0.006683,0.066484,0.073343,117
1,1,0.017153,0.025874,0.001185,0.026018,0.035668,385
2,2,0.024081,0.021618,0.003485,0.040984,0.067957,221
3,3,0.067931,0.056023,0.008944,0.140307,0.172615,41
4,4,0.036662,0.032338,0.006810,0.071832,0.109472,115
5,5,0.064211,0.112625,0.005485,0.142333,0.238563,43
6,6,0.017827,0.013025,0.002336,0.036043,0.042098,340
7,7,0.022617,0.020913,0.002736,0.041387,0.051753,199


In [30]:
t95 = p95(c_data['d_pca'])

In [31]:
c_data['anom_pca_cm'] = 0
c_data.loc[c_data['d_pca']>t95,'anom_pca_cm'] = 1

### Detecting Anomalies - Sensor outages (Category 4) - Suffix _so
Based on EDA - R1

In [32]:
# Type 1: Simultaneous failures of all the sensors - Exact
# Checking for failure correlations
failures = df_c[pollutants].isnull().astype(int).apply(np.prod,axis=1).reset_index().rename(columns={0:'simul_failures'})
failures = failures.groupby(df_c['date'])['simul_failures'].sum().reset_index()
failures = failures[failures['simul_failures']>0]
failures.simul_failures.value_counts()

# This dataframe contains the dates on which all of the sensors failed simultaneously during for at least one hour

simul_failures
1     58
2     16
24    15
3     12
6      5
4      3
5      2
7      2
8      2
9      1
14     1
17     1
18     1
11     1
Name: count, dtype: int64

In [33]:
c_data = c_data.merge(failures,on='date',how='left')

In [34]:
c_data['simul_failures'] = c_data['simul_failures'].fillna(0)
c_data['anom_data_1_so'] = 0
c_data.loc[c_data['simul_failures']>0,'anom_data_1_so'] = 1
c_data

,date,pm2.5,pm10,so2,no2,co,o3,pm2.5_t,pm10_t,so2_t,no2_t,co_t,o3_t,cluster_id,d,anom_cm,pca_1,pca_2,pca_3,cluster_id_pca,d_pca,anom_pca_cm,simul_failures,anom_data_1_so
0,2013-03-01,7.125000,10.750000,11.708333,22.583333,0.429167,63.875000,0.015829,0.021486,0.088785,0.132898,0.141374,0.330555,1,0.008399,0,-0.295134,-0.100661,0.069342,1,0.010237,0,0.0,0
1,2013-03-02,30.750000,42.083333,36.625000,66.666667,0.824917,29.750000,0.061856,0.078781,0.262861,0.381318,0.180522,0.156679,6,0.035934,0,0.019098,-0.182715,0.128209,3,0.027898,0,0.0,0
2,2013-03-03,76.916667,120.541667,61.291667,81.000000,1.620625,19.125000,0.151798,0.222248,0.435191,0.462090,0.259234,0.102542,0,0.012337,0,0.263227,-0.133823,0.228975,5,0.009445,0,0.0,0
3,2013-03-04,22.708333,44.583333,22.869565,46.956522,0.617391,53.750000,0.046189,0.083352,0.166761,0.270247,0.159994,0.278965,1,0.022487,0,-0.132480,-0.095857,0.086555,1,0.022049,0,0.0,0
4,2013-03-05,148.875000,183.791667,93.875000,132.833333,2.357958,68.458333,0.291988,0.337905,0.662829,0.754184,0.332172,0.353908,3,0.152022,1,0.514699,0.214526,0.352351,2,0.055694,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2017-02-24,21.541667,32.625000,16.583333,58.875000,0.575000,50.791667,0.043916,0.061486,0.122843,0.337410,0.155800,0.263892,6,0.030138,0,-0.117616,-0.110316,0.027622,3,0.012180,0,0.0,0
1457,2017-02-25,11.208333,19.708333,6.750000,43.375000,0.420833,65.875000,0.023784,0.037867,0.054144,0.250064,0.140550,0.340745,1,0.008007,0,-0.240703,-0.070382,0.002726,1,0.001156,0,0.0,0
1458,2017-02-26,28.125000,40.708333,10.083333,65.375000,0.720833,48.625000,0.056742,0.076267,0.077432,0.374039,0.170226,0.252852,6,0.022994,0,-0.093136,-0.106215,-0.030477,3,0.007371,0,0.0,0
1459,2017-02-27,71.954545,94.863636,18.809524,98.000000,1.366667,35.272727,0.142131,0.175294,0.138396,0.557889,0.234113,0.184819,6,0.051063,0,0.147566,-0.076993,-0.058546,7,0.033190,0,2.0,1


In [35]:
# Type 2: Simultaneous failures of one or more sensors
# Checking for failure correlations
app_failures = df_c[pollutants].isnull().groupby([df_c['date']]).sum().reset_index()
app_failures[pollutants] = app_failures[pollutants].map(lambda x: 1 if x > 0 else 0)
app_failures['sensors_to_check'] = app_failures[pollutants].apply(lambda x: ''.join(str(k) for k in x),axis=1)
app_failures['n_sensors'] = app_failures[pollutants].apply(sum,axis=1)
app_failures = app_failures[app_failures['n_sensors']>1].reset_index(drop=True)
print('Order for these flags is:',pollutants)
app_failures.sensors_to_check.value_counts()

# Type 1 is actually redundant now

Order for these flags is: ['pm2.5', 'pm10', 'so2', 'no2', 'co', 'o3']


sensors_to_check
111111    121
000011     58
000101     57
001111     41
110000     12
100001     10
001011      7
110001      7
101111      6
000111      5
110100      5
000110      5
100011      4
001010      4
100101      4
001101      4
100010      3
001110      3
001001      3
001100      2
111011      2
100100      2
010011      1
101011      1
111000      1
010001      1
110010      1
110011      1
101001      1
110101      1
Name: count, dtype: int64

In [36]:
c_data = c_data.merge(app_failures[['date','sensors_to_check','n_sensors']],on='date',how='left')
c_data['sensors_to_check'] = c_data['sensors_to_check'].fillna('')
c_data['n_sensors'] = c_data['n_sensors'].fillna(0)
c_data = c_data.rename(columns={'sensors_to_check':'sensors_failing_simultaneously_so', 'n_sensors':'n_sensors_so'})
c_data

,date,pm2.5,pm10,so2,no2,co,o3,pm2.5_t,pm10_t,so2_t,no2_t,co_t,o3_t,cluster_id,d,anom_cm,pca_1,pca_2,pca_3,cluster_id_pca,d_pca,anom_pca_cm,simul_failures,anom_data_1_so,sensors_failing_simultaneously_so,n_sensors_so
0,2013-03-01,7.125000,10.750000,11.708333,22.583333,0.429167,63.875000,0.015829,0.021486,0.088785,0.132898,0.141374,0.330555,1,0.008399,0,-0.295134,-0.100661,0.069342,1,0.010237,0,0.0,0,,0.0
1,2013-03-02,30.750000,42.083333,36.625000,66.666667,0.824917,29.750000,0.061856,0.078781,0.262861,0.381318,0.180522,0.156679,6,0.035934,0,0.019098,-0.182715,0.128209,3,0.027898,0,0.0,0,,0.0
2,2013-03-03,76.916667,120.541667,61.291667,81.000000,1.620625,19.125000,0.151798,0.222248,0.435191,0.462090,0.259234,0.102542,0,0.012337,0,0.263227,-0.133823,0.228975,5,0.009445,0,0.0,0,,0.0
3,2013-03-04,22.708333,44.583333,22.869565,46.956522,0.617391,53.750000,0.046189,0.083352,0.166761,0.270247,0.159994,0.278965,1,0.022487,0,-0.132480,-0.095857,0.086555,1,0.022049,0,0.0,0,001110,3.0
4,2013-03-05,148.875000,183.791667,93.875000,132.833333,2.357958,68.458333,0.291988,0.337905,0.662829,0.754184,0.332172,0.353908,3,0.152022,1,0.514699,0.214526,0.352351,2,0.055694,0,0.0,0,,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2017-02-24,21.541667,32.625000,16.583333,58.875000,0.575000,50.791667,0.043916,0.061486,0.122843,0.337410,0.155800,0.263892,6,0.030138,0,-0.117616,-0.110316,0.027622,3,0.012180,0,0.0,0,,0.0
1457,2017-02-25,11.208333,19.708333,6.750000,43.375000,0.420833,65.875000,0.023784,0.037867,0.054144,0.250064,0.140550,0.340745,1,0.008007,0,-0.240703,-0.070382,0.002726,1,0.001156,0,0.0,0,,0.0
1458,2017-02-26,28.125000,40.708333,10.083333,65.375000,0.720833,48.625000,0.056742,0.076267,0.077432,0.374039,0.170226,0.252852,6,0.022994,0,-0.093136,-0.106215,-0.030477,3,0.007371,0,0.0,0,,0.0
1459,2017-02-27,71.954545,94.863636,18.809524,98.000000,1.366667,35.272727,0.142131,0.175294,0.138396,0.557889,0.234113,0.184819,6,0.051063,0,0.147566,-0.076993,-0.058546,7,0.033190,0,2.0,1,111111,6.0


In [37]:
# Type 3: Too many failures during a day
# Relatively straightforward check. If a single sensor fails for more than N times a day, a flag is raised
col_names = [x + '_so' for x in pollutants]
N = 12

ssf = df_c.isnull().groupby(df_c['date'])[pollutants].sum().reset_index()
ssf[col_names] = ssf[pollutants].map(lambda x: 1 if x >= N else 0)
ssf[ssf[col_names].sum(axis=1)>0].shape

(84, 13)

In [38]:
c_data = c_data.merge(ssf[['date']+col_names],on='date',how='left')
c_data

,date,pm2.5,pm10,so2,no2,co,o3,pm2.5_t,pm10_t,so2_t,no2_t,co_t,o3_t,cluster_id,d,anom_cm,pca_1,pca_2,pca_3,cluster_id_pca,d_pca,anom_pca_cm,simul_failures,anom_data_1_so,sensors_failing_simultaneously_so,n_sensors_so,pm2.5_so,pm10_so,so2_so,no2_so,co_so,o3_so
0,2013-03-01,7.125000,10.750000,11.708333,22.583333,0.429167,63.875000,0.015829,0.021486,0.088785,0.132898,0.141374,0.330555,1,0.008399,0,-0.295134,-0.100661,0.069342,1,0.010237,0,0.0,0,,0.0,0,0,0,0,0,0
1,2013-03-02,30.750000,42.083333,36.625000,66.666667,0.824917,29.750000,0.061856,0.078781,0.262861,0.381318,0.180522,0.156679,6,0.035934,0,0.019098,-0.182715,0.128209,3,0.027898,0,0.0,0,,0.0,0,0,0,0,0,0
2,2013-03-03,76.916667,120.541667,61.291667,81.000000,1.620625,19.125000,0.151798,0.222248,0.435191,0.462090,0.259234,0.102542,0,0.012337,0,0.263227,-0.133823,0.228975,5,0.009445,0,0.0,0,,0.0,0,0,0,0,0,0
3,2013-03-04,22.708333,44.583333,22.869565,46.956522,0.617391,53.750000,0.046189,0.083352,0.166761,0.270247,0.159994,0.278965,1,0.022487,0,-0.132480,-0.095857,0.086555,1,0.022049,0,0.0,0,001110,3.0,0,0,0,0,0,0
4,2013-03-05,148.875000,183.791667,93.875000,132.833333,2.357958,68.458333,0.291988,0.337905,0.662829,0.754184,0.332172,0.353908,3,0.152022,1,0.514699,0.214526,0.352351,2,0.055694,0,0.0,0,,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2017-02-24,21.541667,32.625000,16.583333,58.875000,0.575000,50.791667,0.043916,0.061486,0.122843,0.337410,0.155800,0.263892,6,0.030138,0,-0.117616,-0.110316,0.027622,3,0.012180,0,0.0,0,,0.0,0,0,0,0,0,0
1457,2017-02-25,11.208333,19.708333,6.750000,43.375000,0.420833,65.875000,0.023784,0.037867,0.054144,0.250064,0.140550,0.340745,1,0.008007,0,-0.240703,-0.070382,0.002726,1,0.001156,0,0.0,0,,0.0,0,0,0,0,0,0
1458,2017-02-26,28.125000,40.708333,10.083333,65.375000,0.720833,48.625000,0.056742,0.076267,0.077432,0.374039,0.170226,0.252852,6,0.022994,0,-0.093136,-0.106215,-0.030477,3,0.007371,0,0.0,0,,0.0,0,0,0,0,0,0
1459,2017-02-27,71.954545,94.863636,18.809524,98.000000,1.366667,35.272727,0.142131,0.175294,0.138396,0.557889,0.234113,0.184819,6,0.051063,0,0.147566,-0.076993,-0.058546,7,0.033190,0,2.0,1,111111,6.0,0,0,0,0,0,0


### Detecting Anomalies - Deviations from hypotheses/known facts (Category 2) - Suffix _dev
Based on EDA - R5

In [39]:
# 1
# PM 2.5 concentrations should always be lower than PM 10
df_c['pm_conc_anom_dev'] = (df_c['pm2.5'] > df_c['pm10']).astype(int)
df_c.head()

,no,year,month,day,hour,pm2.5,pm10,so2,no2,co,o3,temp,pres,dewp,rain,wd,wspm,station,ts,date,pm2.5_si,pm10_si,so2_si,no2_si,co_si,o3_si,aqi,dominant,wd_angle,pm2.5_st,pm10_st,so2_st,no2_st,co_st,o3_st,pm_conc_anom_dev
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,0.3,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin,2013-03-01 00:00:00,2013-03-01,6.666667,4.0,5.00,8.75,15.0,77.0,77.0,o3_si,337.5,0,0,0,0,0,0,0
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,0.3,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin,2013-03-01 01:00:00,2013-03-01,13.333333,8.0,5.00,8.75,15.0,77.0,77.0,o3_si,0.0,0,0,0,0,0,0,0
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,0.3,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin,2013-03-01 02:00:00,2013-03-01,11.666667,7.0,6.25,12.50,15.0,73.0,73.0,o3_si,337.5,0,0,0,0,0,0,0
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,0.3,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin,2013-03-01 03:00:00,2013-03-01,10.000000,6.0,13.75,13.75,15.0,72.0,72.0,o3_si,315.0,0,0,0,0,0,0,0
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,0.3,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin,2013-03-01 04:00:00,2013-03-01,5.000000,3.0,15.00,15.00,15.0,72.0,72.0,o3_si,0.0,0,0,0,0,0,0,0


In [40]:
# 2
# PM10 concentrations should be correlated with the wind speed. In the absence of wind PM10 readings are higher
# https://www.researchgate.net/publication/341444279_Effect_of_wind_speed_on_the_level_of_particulate_matter_PM10_concentration_in_atmospheric_air_during_winter_season_in_vicinity_of_large_combustion_plant
p99 = lambda x: np.nanpercentile(x,99)
p90 = lambda x: np.nanpercentile(x,90)
p5 = lambda x: np.nanpercentile(x,5)

ws_mod = df_c['wspm'].fillna(-1).astype(int).apply(lambda x: 6 if x >= 6 else x) # Thresholding the wind speed values - higher values do not have a lot of data
ws_mod= df_c.groupby(ws_mod)['pm10'].aggregate(['mean','std','count',p5,p90,p99]).reset_index()
ws_mod = ws_mod.iloc[:,[0,-1]]
ws_mod.columns = ['ws_mod','p99_temp']

In [41]:
ws_vals = df_c[['wspm']].drop_duplicates().fillna(-1)
ws_vals['ws_mod'] = ws_vals['wspm'].fillna(-1).astype(int).apply(lambda x: 6 if x >= 6 else x)

In [42]:
df_c = df_c.merge(ws_vals,on='wspm',how='left')
df_c.head()

,no,year,month,day,hour,pm2.5,pm10,so2,no2,co,o3,temp,pres,dewp,rain,wd,wspm,station,ts,date,pm2.5_si,pm10_si,so2_si,no2_si,co_si,o3_si,aqi,dominant,wd_angle,pm2.5_st,pm10_st,so2_st,no2_st,co_st,o3_st,pm_conc_anom_dev,ws_mod
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,0.3,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin,2013-03-01 00:00:00,2013-03-01,6.666667,4.0,5.00,8.75,15.0,77.0,77.0,o3_si,337.5,0,0,0,0,0,0,0,4.0
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,0.3,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin,2013-03-01 01:00:00,2013-03-01,13.333333,8.0,5.00,8.75,15.0,77.0,77.0,o3_si,0.0,0,0,0,0,0,0,0,4.0
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,0.3,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin,2013-03-01 02:00:00,2013-03-01,11.666667,7.0,6.25,12.50,15.0,73.0,73.0,o3_si,337.5,0,0,0,0,0,0,0,5.0
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,0.3,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin,2013-03-01 03:00:00,2013-03-01,10.000000,6.0,13.75,13.75,15.0,72.0,72.0,o3_si,315.0,0,0,0,0,0,0,0,3.0
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,0.3,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin,2013-03-01 04:00:00,2013-03-01,5.000000,3.0,15.00,15.00,15.0,72.0,72.0,o3_si,0.0,0,0,0,0,0,0,0,2.0


In [43]:
df_c = df_c.merge(ws_mod,on='ws_mod',how='left').drop(columns='ws_mod')
df_c['pm10_ws_anom_dev'] = 0
df_c.loc[df_c['pm10']>df_c['p99_temp'],'pm10_ws_anom_dev'] = 1
df_c = df_c.drop(columns='p99_temp')
df_c.head()

,no,year,month,day,hour,pm2.5,pm10,so2,no2,co,o3,temp,pres,dewp,rain,wd,wspm,station,ts,date,pm2.5_si,pm10_si,so2_si,no2_si,co_si,o3_si,aqi,dominant,wd_angle,pm2.5_st,pm10_st,so2_st,no2_st,co_st,o3_st,pm_conc_anom_dev,pm10_ws_anom_dev
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,0.3,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin,2013-03-01 00:00:00,2013-03-01,6.666667,4.0,5.00,8.75,15.0,77.0,77.0,o3_si,337.5,0,0,0,0,0,0,0,0
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,0.3,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin,2013-03-01 01:00:00,2013-03-01,13.333333,8.0,5.00,8.75,15.0,77.0,77.0,o3_si,0.0,0,0,0,0,0,0,0,0
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,0.3,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin,2013-03-01 02:00:00,2013-03-01,11.666667,7.0,6.25,12.50,15.0,73.0,73.0,o3_si,337.5,0,0,0,0,0,0,0,0
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,0.3,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin,2013-03-01 03:00:00,2013-03-01,10.000000,6.0,13.75,13.75,15.0,72.0,72.0,o3_si,315.0,0,0,0,0,0,0,0,0
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,0.3,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin,2013-03-01 04:00:00,2013-03-01,5.000000,3.0,15.00,15.00,15.0,72.0,72.0,o3_si,0.0,0,0,0,0,0,0,0,0


### Detecting Anomalies - Detecting Latent Patterns (Category 5) - Suffix _lat

In [55]:
# Local Outlier Factor
lof_df = df_c[pollutants+covars+['year','month','hour']].fillna(-1)
lof = LocalOutlierFactor(n_neighbors=120) # High number of neighbors to capture samples across both time and sensor variables
df_c['lof_label_lat'] = lof.fit_predict(lof_df)
df_c['lof_label_lat'].value_counts()

lof_label_lat
 1    34505
-1      559
Name: count, dtype: int64

In [56]:
# Isolation Forest
isof = IsolationForest(random_state=14)
isof_df = df_c[pollutants+covars+['month','hour','wd_angle']].fillna(-1) # Not including year because the trend is not that significant
isof = isof.fit(isof_df)
df_c['isof_label_lat'] = isof.predict(isof_df)
df_c['isof_label_lat'].value_counts()

isof_label_lat
 1    30897
-1     4167
Name: count, dtype: int64